In [152]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

# Word lists and lexicons in nltk: https://www.nltk.org/howto/corpus.html#word-lists-and-lexicons
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()
sia.polarity_scores("This is a really great tweet!")

# Output:
# [nltk_data] Downloading package vader_lexicon to /root/nltk_data...
# [nltk_data]   Package vader_lexicon is already up-to-date!
# {'compound': 0.6893, 'neg': 0.0, 'neu': 0.461, 'pos': 0.539}

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\ZyroY\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


{'neg': 0.0, 'neu': 0.461, 'pos': 0.539, 'compound': 0.6893}

In [153]:
import pandas as pd
import numpy as np
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Load the dataset
df = pd.read_csv('tw_tweets_users_media_places.csv')
df.head()




,tweet_id,text,context_annotations_count,count_annotations,count_cashtags,count_hashtags,count_mentions,count_urls,created_at_tweet,lang,...,location,protected,verified,media_type,height,width,preview_image_url,country,name_place,place_type
0,1440484799970304000,This was my grandson this morning (w/autism)! ...,1,0.0,0.0,0.0,0.0,1.0,2021-09-22T01:15:13.000Z,en,...,"Victoria, BC",False,False,photo,405,813,https://pbs.twimg.com/media/E_2hSs4UcAAIOK5.jpg,Canada,Langford,city
1,1439618825171963904,Wow!! Been into #York for the first time since...,2,2.0,0.0,3.0,0.0,1.0,2021-09-19T15:54:09.000Z,en,...,"Hessay, York",False,False,photo,2048,1536,https://pbs.twimg.com/media/E_qNsE1X0AQmoK_.jpg,United Kingdom,Hessay,city
2,1248872872837332992,Sad number of ppl who lost life due to covid-1...,3,0.0,0.0,0.0,0.0,1.0,2020-04-11T07:17:50.000Z,en,...,"Maidstone, South East",False,False,photo,288,278,https://pbs.twimg.com/media/EVTjQcoXsAAlrfq.jpg,United Kingdom,Maidstone,city
3,1250729294051053568,Webinar now available‘Staying healthy at home ...,1,2.0,0.0,3.0,0.0,2.0,2020-04-16T10:14:35.000Z,en,...,"Maidstone, South East",False,False,photo,2048,2048,https://pbs.twimg.com/media/EVt7pYTXkAMGzxj.jpg,United Kingdom,Maidstone,city
4,1249612131433095168,Webinar now available‘Staying healthy at home ...,1,2.0,0.0,3.0,0.0,2.0,2020-04-13T08:15:23.000Z,en,...,"Maidstone, South East",False,False,photo,2048,2048,https://pbs.twimg.com/media/EVeDlp7X0AMuN6X.jpg,United Kingdom,Maidstone,city


In [154]:
# First, check the data types
print("\nData types in the dataset:")
print(df.dtypes)
# Select only numerical columns


Data types in the dataset:
tweet_id                       int64
text                          object
context_annotations_count      int64
count_annotations            float64
count_cashtags               float64
count_hashtags               float64
count_mentions               float64
count_urls                   float64
created_at_tweet              object
lang                          object
likes                          int64
quotes                         int64
referenced_tweet_count         int64
replies                        int64
reply_settings                object
retweets                       int64
source                        object
terms                         object
username                      object
created_at_author             object
followers_count                int64
following_count                int64
tweet_count                    int64
listed_count                   int64
location                      object
protected                       bool
verified  

In [155]:
from nltk.sentiment import SentimentIntensityAnalyzer

# Initialize sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Ensure 'text' column is string and handle NaNs
df['text'] = df['text'].astype(str).fillna('')

# Initialize sentiment columns
df['sentiment_overall'] = 0.0
df['sentiment_neg'] = 0.0
df['sentiment_neu'] = 0.0
df['sentiment_pos'] = 0.0

# Perform sentiment analysis
for row in df.itertuples():
    sentiment = sia.polarity_scores(row.text)
    df.loc[row.Index, 'sentiment_overall'] = sentiment['compound']
    df.loc[row.Index, 'sentiment_neg'] = sentiment['neg']
    df.loc[row.Index, 'sentiment_neu'] = sentiment['neu']
    df.loc[row.Index, 'sentiment_pos'] = sentiment['pos']


In [156]:
df = df.select_dtypes(include=['number', 'float64', 'int64'])

In [157]:
import statsmodels.api as sm


y = df['retweets']
X = df.drop(columns=['retweets', 'sentiment_overall', 'context_annotations_count', 'count_annotations', 'count_cashtags', 'count_hashtags', 
                     'count_mentions', 'count_urls', 'referenced_tweet_count', 'listed_count', 'width', 'height', 'tweet_count', 
                     'sentiment_neu', 'sentiment_neg', 'sentiment_pos', 'likes', 'quotes', 'replies', 'following_count']).assign(const=1)

print(sm.OLS(y, X).fit().summary())

# Output:
# R-squared: 0.246

                            OLS Regression Results                            
Dep. Variable:               retweets   R-squared:                       0.058
Model:                            OLS   Adj. R-squared:                  0.056
Method:                 Least Squares   F-statistic:                     32.46
Date:                Tue, 08 Apr 2025   Prob (F-statistic):           2.02e-08
Time:                        20:47:50   Log-Likelihood:                -2333.5
No. Observations:                 534   AIC:                             4671.
Df Residuals:                     532   BIC:                             4680.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
tweet_id         3.061e-18   6.41e-19     

#### retweets=(−1.601 * 10^-12) + ((3.061 * 10^-18) * tweet_id) + ((8.242 * 10^-5) * followers_count)
#### Our model goes as above

#### The significant predictors of retweets are **followers_count** (p < 0.001) and **tweet_id** (p < 0.001). More followers correlate with more retweets, while tweet_id likely reflects time-based effects. Sentiment variables were not significant, as they were not included in the final model.

In [158]:
df = pd.read_csv('tw_tweets_users_media_places.csv')
df.head()


,tweet_id,text,context_annotations_count,count_annotations,count_cashtags,count_hashtags,count_mentions,count_urls,created_at_tweet,lang,...,location,protected,verified,media_type,height,width,preview_image_url,country,name_place,place_type
0,1440484799970304000,This was my grandson this morning (w/autism)! ...,1,0.0,0.0,0.0,0.0,1.0,2021-09-22T01:15:13.000Z,en,...,"Victoria, BC",False,False,photo,405,813,https://pbs.twimg.com/media/E_2hSs4UcAAIOK5.jpg,Canada,Langford,city
1,1439618825171963904,Wow!! Been into #York for the first time since...,2,2.0,0.0,3.0,0.0,1.0,2021-09-19T15:54:09.000Z,en,...,"Hessay, York",False,False,photo,2048,1536,https://pbs.twimg.com/media/E_qNsE1X0AQmoK_.jpg,United Kingdom,Hessay,city
2,1248872872837332992,Sad number of ppl who lost life due to covid-1...,3,0.0,0.0,0.0,0.0,1.0,2020-04-11T07:17:50.000Z,en,...,"Maidstone, South East",False,False,photo,288,278,https://pbs.twimg.com/media/EVTjQcoXsAAlrfq.jpg,United Kingdom,Maidstone,city
3,1250729294051053568,Webinar now available‘Staying healthy at home ...,1,2.0,0.0,3.0,0.0,2.0,2020-04-16T10:14:35.000Z,en,...,"Maidstone, South East",False,False,photo,2048,2048,https://pbs.twimg.com/media/EVt7pYTXkAMGzxj.jpg,United Kingdom,Maidstone,city
4,1249612131433095168,Webinar now available‘Staying healthy at home ...,1,2.0,0.0,3.0,0.0,2.0,2020-04-13T08:15:23.000Z,en,...,"Maidstone, South East",False,False,photo,2048,2048,https://pbs.twimg.com/media/EVeDlp7X0AMuN6X.jpg,United Kingdom,Maidstone,city


In [159]:
import sys
import re
import numpy as np
from pprint import pprint
    
# Gensim
import gensim, logging, warnings
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
    
# NLTK Stop words
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 'co']) # After reviewing the LDA, return to add words that you want to eliminate

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ZyroY\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [160]:
# Print total number of tweets
print(f'Total tweets: {len(df)}')

# Remove retweets by checking if 'RT @' appears in the 'text' column
df_no_retweets = df[~df['text'].str.contains("RT @", na=False)]

# Print number of original tweets (after removing retweets)
print(f'Original tweets: {len(df)}')


Total tweets: 534
Original tweets: 534


In [161]:
def sent_to_words(sentences):
    for sent in sentences:
        sent = re.sub(r'\S*@\S*\s?', '', sent)  # remove emails
        sent = re.sub(r'\s+', ' ', sent)  # remove newline chars
        sent = re.sub(r"'", "", sent)  # remove single quotes
        sent = gensim.utils.simple_preprocess(str(sent), deacc=True)
        yield sent

# Convert each tweet to a list of cleaned words and add to a master list
data = df_no_retweets['text'].values.tolist()  # Correct column access for df_no_retweets
data_words = list(sent_to_words(data))

# Print the first 5 cleaned tweet word lists
for tweet in data_words[:5]:
    print(tweet)

['this', 'was', 'my', 'grandson', 'this', 'morning', 'autism', 'apparently', 'he', 'is', 'going', 'through', 'bout', 'of', 'bronchitis', 'amp', 'pneumonia', 'he', 'always', 'gets', 'it', 'every', 'year', 'my', 'daughter', 'said', 'that', 'she', 'is', 'sure', 'she', 'will', 'have', 'to', 'take', 'him', 'to', 'the', 'hospital', 'tonight', 'but', 'if', 'they', 'try', 'to', 'test', 'him', 'for', 'covid', 'she', 'will', 'walk', 'out', 'https', 'co', 'krqjit']
['wow', 'been', 'into', 'york', 'for', 'the', 'first', 'time', 'since', 'christmas', 'shopping', 'in', 'dec', 'took', 'our', 'daughter', 'cerys', 'to', 'the', 'catcafe', 'to', 'celebrate', 'her', 'th', 'birthday', 'perfectly', 'relaxed', 'quiet', 'covid', 'unfriendly', 'and', 'autism', 'friendly', 'she', 'was', 'in', 'her', 'happy', 'place', 'https', 'co', 'qzck', 'fze']
['sad', 'number', 'of', 'ppl', 'who', 'lost', 'life', 'due', 'to', 'covid', 'is', 'far', 'more', 'than', 'the', 'number', 'that', 'it', 'says', 'on', 'the', 'news', 't

In [162]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
# !python -m spacy download en_core_web_sm
import spacy

ImportError: cannot import name util

In [ ]:
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    # remove stop words using list comprehension
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    # add bigrams and trigrams using list comprehension
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
            
    texts_out = []
    nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])    # Load spacy, but we don't need the parser or NER (named entity extraction) modules
        
    # perform lemmatization and another round of stopword removal to catch any that were created by the lemmatization process
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
             
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
      
    return texts_out


In [ ]:
data_ready = process_words(data_words)  # processed Text Data!
for tweet in data_ready[:5]:
    print(tweet)

AttributeError: module 'spacy_loggers' has no attribute 'load'

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_ready)
        
for row in id2word.iteritems():
    print(row)

NameError: name 'data_ready' is not defined

Channels:
 - conda-forge
 - defaults
Platform: win-64
Solving environment: ...working... failed


warning  libmamba Added empty dependency for problem type SOLVER_RULE_UPDATE

LibMambaUnsatisfiableError: Encountered problems while solving:
  - package spacy-3.0.0-py36h9a8d428_0 requires python >=3.6,<3.7.0a0, but none of the providers can be installed

Could not solve for environment specs
The following packages are incompatible
├─ pin-1 is installable and it requires
│  └─ python 3.12.* , which can be installed;
└─ spacy 3.0**  is not installable because there are no viable options
   ├─ spacy [3.0.0|3.0.1|...|3.0.6] would require
   │  └─ python >=3.6,<3.7.0a0 , which conflicts with any installable versions previously reported;
   ├─ spacy [3.0.0|3.0.1|...|3.0.6] would require
   │  └─ python >=3.7,<3.8.0a0 , which conflicts with any installable versions previously reported;
   ├─ spacy [3.0.0|3.0.1|...|3.0.6] would require
   │  └─ python >=3.8,<3.9.0a0 , which conflicts with any installable versions previously reported;
   └─ spacy [3.0.0|3.0.1|...|3.0.6] would require
      └─

In [ ]:
topics = 4
            
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, # This is the corpus we created above
                                                    id2word=id2word, # This is the dictionary we created above
                                                    num_topics=topics,
                                                    random_state=12345,
                                                    chunksize=100,
                                                    passes=5,
                                                    per_word_topics=True)
            
ldatopics = lda_model.show_topics(formatted=False)
pprint(lda_model.print_topics())

In [ ]:
df_fit = pd.DataFrame(columns=['topics', 'perplexity', 'coherence'])

for n in range(3,10):
    # Fit LDA model
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, 
                                                      id2word=id2word, 
                                                      num_topics=n, 
                                                      random_state=12345,
                                                      chunksize=100, 
                                                      passes=5,
                                                      per_word_topics=True)
        
    # Generate fit metrics
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_ready, dictionary=id2word, coherence='c_v')
    # Add metrics to df_fit
    df_fit.loc[str(n - 3)] = [n, round(lda_model.log_perplexity(corpus), 3), round(coherence_model_lda.get_coherence(), 3)]
        
df_fit

In [ ]:
import seaborn as sns
        
df_fit['diff'] = abs(df_fit.coherence - df_fit.perplexity)
sns.lineplot(x='topics', y='diff', data=df_fit, ci=None, marker='o');

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, 
                                              id2word=id2word, 
                                              num_topics=8, 
                                              random_state=1,
                                              chunksize=100, 
                                              passes=5,
                                              per_word_topics=True)

#NOTICE THE ABOVE CODE IS SLIGHTLY DIFFERENT FROM YOUR BOOK

df_topics = df_tweets.copy()
        
num_topics = len(lda_model.get_topics()) # store the number of topics from the last model
for col in range(num_topics): # generate a new column for each topic
    df_topics[f'topic_{col + 1}'] = 0.0
          
        # Store the topic score and dominant topic
for i, words in enumerate(data_ready):
    doc = lda_model[id2word.doc2bow(words)] # generate a corpus for this document set of words
          
    for j, score in enumerate(doc[0]): # for each document in the corpus
            # Get the topic score and store it in the appropriate column
        df_topics.iat[i, (len(df_topics.columns) - ((num_topics) - score[0]))] = score[1]
        
df_topics.head()
        

In [ ]:
# Display setting to show more characters in column
pd.options.display.max_colwidth = 100
        
        # Create the output DataFrame
df_representative_tweets = pd.DataFrame(columns=df_topics.columns)
        
        # Iterate through each topic
for n in range(1, num_topics + 1):
          # Copy the row from the original df with the highest topic score into the new df
    df_representative_tweets.loc['topic_' + str(n)] = df_topics.loc[df_topics['topic_' + str(n)].idxmax()]
        
df_representative_tweets

NameError: name 'df_topics' is not defined

In [ ]:
# Bar chart of word counts for each topic
from collections import Counter
import matplotlib.colors as mcolors, math

topics = lda_model.show_topics(formatted=False)
data_flat = [w for w_list in data_ready for w in w_list]
counter = Counter(data_flat)

out = []
for i, topic in topics:
  for word, weight in topic:
    out.append([word, i + 1, weight, counter[word]])

df_temp = pd.DataFrame(out, columns=['word', 'topic_id', 'importance', 'word_count'])

# Plot Word Count and Weights of Topic Keywords
matrix_size = math.ceil(num_topics**(1/2))  # Computes the n by n number of plots to generate
fig, axes = plt.subplots(matrix_size, matrix_size, figsize=(20,20), sharey=True, dpi=160)
cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]
for i, ax in enumerate(axes.flatten()):
  ax.bar(x='word', height="word_count", data=df_temp.loc[df_temp.topic_id==i+1, :], color=cols[i+1], width=0.5, alpha=0.3, label='Word Count')
  ax_twin = ax.twinx()
  ax_twin.bar(x='word', height="importance", data=df_temp.loc[df_temp.topic_id==i+1, :], color=cols[i+1], width=0.2, label='Weights')
  ax.set_ylabel('Word Count', color=cols[i+1])
  ax.set_title('Topic: ' + str(i + 1), color=cols[i+1], fontsize=16)
  ax.tick_params(axis='y', left=False)
  ax.set_xticks(ax.get_xticks())
  ax.set_xticklabels(df_temp.loc[df_temp.topic_id==i+1, 'word'], rotation=30, horizontalalignment= 'right')
  ax.legend(loc='upper center'); ax_twin.legend(loc='upper right')
  if i >= len(topics): # Turn off the unneeded subplots
    ax.axis('off')
    ax.title.set_visible(False)
    ax_twin.axis('off')
    ax.legend().set_visible(False)
    ax_twin.legend().set_visible(False)

fig.tight_layout(w_pad=2)
fig.suptitle('Word Count and Importance of Topic Keywords', fontsize=20, y=1.03)
plt.show()